## Importing Libraries

This section imports essential Python libraries:

- **`socket`**: Enables network communication and handling server-client connections.
- **`threading`**: Provides tools for multithreaded operations, allowing the program to handle multiple tasks simultaneously.
- **`time`**: Offers utilities for managing delays and timestamps.
- **`math`**: Supplies mathematical functions like square roots and trigonometric calculations.
- **`logging`**: Configures and manages logs for debugging and tracking events.
- **`heapq`**: Implements an efficient priority queue for sorting and managing tasks.


In [ ]:
import socket
import threading
import time
import math
import logging
import heapq


## Constants Initialization

This section initializes constants that define important parameters for the script:

- **`HOST`**: The server listens on all available network interfaces (`0.0.0.0`).
- **`PORT`**: Port number (`13000`) where the server accepts incoming connections.
- **`JobId`**: A unique identifier for tasks, initialized to `50` for the Robotino fleet.
- **`MAX_BUFFER_SIZE`**: The maximum size of incoming data packets (4096 bytes).
- **`CURRENT_ROBOTINO_STATE`**: Placeholder for storing the current state of Robotinos (set to `None` initially).
- **`BATTERY_MINIMUM_PERCENT`**: The minimum battery percentage threshold (20%) below which Robotinos need to be charged.


In [ ]:
HOST = '0.0.0.0'  # Host to listen on
PORT = 13000  # Port for server to listen on

JobId = 50  # Start with JobId 1 for Robotino Fleet
MAX_BUFFER_SIZE = 4096  # Maximum buffer size for incoming messages
CURRENT_ROBOTINO_STATE = None
BATTERY_MINIMUM_PERCENT = 20


## Logger Configuration

This section sets up a logger for debugging and tracking events. The logger outputs messages to both:

1. **A file**: Logs are saved in a file named `charging_selection.log`.
2. **The console**: Logs are displayed on the terminal.

The log format includes the following details:
- **Timestamp** of the log event.
- **Log level** (e.g., INFO, DEBUG, ERROR).
- **Thread name** for identifying concurrent operations.
- **Message** describing the event.

The logger is initialize


In [ ]:
# Configure the logger
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] [%(threadName)s] %(message)s",
    handlers=[
        logging.FileHandler("charging_selection.log"),  # Logs to a file
        logging.StreamHandler()  # Logs to the console
    ]
)
logging.info("initialized logging")


## Charger and Robotino Configurations

This section defines configuration dictionaries for chargers and Robotinos:

### **Charger Configurations**
- **`charger_configurations`**: A dictionary containing charger IDs as keys. Each charger has:
  - **Coordinates (`X`, `Y`)**: Represents the charger's position.
  - **Type**: Indicates the type of charger (e.g., 3 or 4).

### **Robotino Configurations**
- **`robotino_configurations`**: A dictionary containing Robotino IDs as keys. Each Robotino has:
  - **Type**: Indicates the Robotino's model type (e.g., 3 or 4).

These configurations are used to match Robotinos to compatible chargers and to calculate distances for navigation.


In [ ]:
# Charger locations as a dictionary with their coordinates and types
charger_configurations = {
    11: {"X": -15.262, "Y": 0.853, "type": 3},
    12: {"X": -14.215, "Y": 0.830, "type": 3},
    13: {"X": -0.162, "Y": 1.525, "type": 3},
    14: {"X": -8.677, "Y": -0.988, "type": 3},
    17: {"X": -0.504, "Y": -0.949, "type": 4},
    18: {"X": -0.276, "Y": 0.067, "type": 4},
}

robotino_configurations = {
    20: {"type": 3},
    21: {"type": 3},
    22: {"type": 3},
    23: {"type": 3},
    24: {"type": 4},
    25: {"type": 4},
}

# Fleet state dictionary: To store robot battery percentages and other info
fleet_state = {}


## **Robotino Categorization and Priority Queues**  
This section categorizes Robotinos into various states and initializes priority queues:  

### **Categories**  
- **active_robotinos**: Robotinos actively performing tasks.  
- **inactive_robotinos**: Robotinos not currently performing tasks.  
- **drained_robotinos**: Robotinos with drained batteries.  
- **robots_moving_to_charger**: Robotinos assigned to move to a charger.  

### **Priority Queues**  
- **operational_queue**: Contains Robotinos ready for tasks.  
- **charging_queue**: Contains Robotinos currently charging.  
- **heapq.heapify**: Ensures priority queue behavior for efficient management.  

### **Chargers List**  
- **chargers**: A placeholder list for actual charger objects.  


In [ ]:
# Lists to categorize Robotinos
active_robotinos = []
inactive_robotinos = []
drained_robotinos = []
robots_moving_to_charger = []

# Initialize queues for operational and charging Robotinos
operational_queue = [20, 21, 24]  # Robotino IDs: 2 version 3 and 1 version 4
charging_queue = [22, 23, 25]  # Robotino IDs: 2 version 3 and 1 version 4

# Use heapq to maintain priority queue behavior
heapq.heapify(operational_queue)
heapq.heapify(charging_queue)

# Simulated list of chargers (update with actual charger objects if available)
chargers = []  # Example: chargers = [Charger(11), Charger(12), ...]


## **Function: calculate_distance**  
### **Purpose**  
Calculates the Euclidean distance between two points in a 2D plane.  

### **Parameters**  
- **x1, y1**: Coordinates of the first point.  
- **x2, y2**: Coordinates of the second point.  

### **Returns**  
- **Distance**: A floating-point value representing the distance.  

### **Formula**  
\[
\text{distance} = \sqrt{(x2 - x1)^2 + (y2 - y1)^2}
\]


In [ ]:
def calculate_distance(x1, y1, x2, y2):
    """
    Calculates the Euclidean distance between two points (x1, y1) and (x2, y2).
    """
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)


## **Function: is_charger_occupied**  
### **Purpose**  
Checks whether a specific charger is currently occupied by any Robotino.  

### **Parameters**  
- **charger_id**: The unique ID of the charger to check.  

### **Logic**  
1. Retrieves the coordinates of the charger using its ID.  
2. Iterates through the **fleet_state** to check the positions of Robotinos.  
3. If a Robotino is within **20 cm** of the charger, it is marked as occupied.  

### **Returns**  
- **True**: If the charger is occupied.  
- **False**: If the charger is unoccupied.  

### **Logging**  
Logs detailed messages for validation and debugging purposes.  


In [ ]:
def is_charger_occupied(charger_id):
    """
    Determines if a charger is occupied based on the current fleet state.
    A charger is considered occupied if a Robotino is within 20 cm of the charger.
    """
    logging.info(f"checking if charger {charger_id} is occupied")
    if charger_id not in charger_configurations:
        logging.info(f"Charger ID {charger_id} is not valid.")
        return False

    charger_coords = charger_configurations[charger_id]
    charger_x, charger_y = charger_coords["X"], charger_coords["Y"]

    for robot_id, robot_info in fleet_state.items():
        robot_x, robot_y = robot_info['x'], robot_info['y']
        distance = calculate_distance(charger_x, charger_y, robot_x, robot_y)
        if distance <= 0.2:  # 20 cm threshold
            logging.info(f"charger is in use {charger_id} by Robotino {robot_id}.")
            return True  # Charger is occupied
    logging.info(f"charger {charger_id} is not occupied.")
    return False  # Charger is not occupied


## **Function: find_closest_free_charger**  
### **Purpose**  
Identifies the nearest unoccupied charger for a specific Robotino based on its position.  

### **Parameters**  
- **robot_id**: The unique ID of the Robotino.  

### **Logic**  
1. Checks if the Robotino exists in the **fleet_state**.  
2. Retrieves the Robotino's current coordinates.  
3. Iterates through all chargers:  
   - Skips occupied chargers.  
   - Skips chargers incompatible with the Robotino's type.  
4. Calculates the distance to each available charger and selects the closest one.  

### **Returns**  
- **Closest charger ID**: If a suitable charger is found.  
- **None**: If no charger is available.  

### **Logging**  
Logs information about the selection process, including skipped chargers and final decisions.  


In [ ]:
def find_closest_free_charger(robot_id):
    """
    Finds the closest unoccupied charger for the given Robotino ID based on its position.
    Uses dynamic occupancy checking based on fleet state.
    """
    if robot_id not in fleet_state:
        logging.error(f"skipping search for charger of Robot {robot_id}. Cause: Robot not found in fleet state.")
        return None

    robot_x = fleet_state[robot_id]['x']
    robot_y = fleet_state[robot_id]['y']
    logging.info(f"searching charger for robot: {robot_id} (coordinates: {robot_x}, {robot_y})")

    closest_charger = None
    shortest_distance = float('inf')

    for charger_id, charger_config in charger_configurations.items():
        if is_charger_occupied(charger_id):
            logging.info(f"skipping charger {charger_id}. Cause: occupied charger.")
            continue  # Skip occupied chargers

        if charger_configurations[charger_id]['type'] != robotino_configurations[robot_id]['type']:
            logging.info(f"skipping charger {charger_id}. Cause: "
                         f"charger model{charger_configurations[charger_id]['type']} does not match "
                         f"robotino model{robotino_configurations[robot_id]['type']}.")
            continue  # skip wrong type

        distance = calculate_distance(robot_x, robot_y, charger_config["X"], charger_config["Y"])
        logging.info(
            f"coords charger: {charger_config['X']}, {charger_config['Y']} ||coords robot: {robot_x}, {robot_y} ")
        if distance < shortest_distance:
            closest_charger = charger_id
            logging.info(f"selected charger {charger_id} with distance {distance} as new candidate")
            shortest_distance = distance

    if closest_charger is None:
        logging.error(f"no charger found for robotino{robot_id}.")
    else:
        logging.info(f"closest charger is {closest_charger}")
    return closest_charger


## **Function: send_robot_to_closest_charger**  
### **Purpose**  
Generates a command to send a Robotino to its nearest unoccupied charger.  

### **Parameters**  
- **robot_id**: The unique ID of the Robotino to be sent.  

### **Logic**  
1. Verifies that the Robotino exists in **fleet_state**.  
2. Calls **find_closest_free_charger** to identify the nearest suitable charger.  
3. Constructs a command string to instruct the Robotino to move to the charger.  
4. Increments the global **JobId** counter for unique task assignment.  

### **Returns**  
- **Command string**: A formatted instruction to send the Robotino to the charger.  
- **None**: If no charger is available or the Robotino is not found.  

### **Logging/Output**  
- Outputs the command or error messages to the console.  


In [ ]:
def send_robot_to_closest_charger(robot_id):
    """
    Generates a command to send the robot to the closest unoccupied charger.
    """
    global JobId

    if robot_id not in fleet_state:
        print(f"Robot ID {robot_id} not found in fleet state.")
        return None

    closest_charger = find_closest_free_charger(robot_id)

    if closest_charger is None:
        print("No available chargers.")
        return None

    JobId += 1
    command = (
        f"PushJob GotoPosition {JobId} 1 {robot_id} {closest_charger}\n"
    )
    print(f"Sending robot {robot_id} to closest charger (ID: {closest_charger}): {command}")
    return command


## **Function: send_all_robots_to_closest_chargers**  
### **Purpose**  
Sends all active Robotinos in the fleet to their respective nearest unoccupied chargers.  

### **Logic**  
1. Iterates over all Robotinos in the **fleet_state** dictionary.  
2. Calls **send_robot_to_closest_charger** for each Robotino to generate and execute commands.  

### **Returns**  
- No direct return. Commands are generated and printed for each Robotino.  

### **Output**  
Prints the commands for each Robotino to the console.  


In [ ]:
def send_all_robots_to_closest_chargers():
    """
    Sends all Robotinos in the fleet to their closest unoccupied chargers.
    """
    for robot_id in fleet_state.keys():
        command = send_robot_to_closest_charger(robot_id)
        if command:
            print(command)


## **Function: send_robot_to_dock**  
### **Purpose**  
Generates a command to dock a Robotino to a charger.  

### **Parameters**  
- **robot_id**: The unique ID of the Robotino to dock.  

### **Logic**  
1. Increments the global **JobId** to ensure unique task identification.  
2. Constructs a command string instructing the Robotino to dock.  

### **Returns**  
- **Command string**: The docking instruction.  

### **Output**  
Prints the docking command to the console.  


In [ ]:
def send_all_robots_to_dock():
    """
    Sends all Robotinos in the fleet to dock at their chargers.
    """
    for robot_id in fleet_state.keys():
        command = send_robot_to_dock(robot_id)
        if command:
            print(command)


## **Function: send_robot_to_position**  
### **Purpose**  
Generates a command to move a Robotino to a specific position.  

### **Parameters**  
- **robot_id**: The unique ID of the Robotino.  
- **target_x, target_y**: Coordinates of the target position.  

### **Logic**  
1. Verifies that the Robotino exists in the **fleet_state** dictionary.  
2. Increments the global **JobId** to maintain unique task IDs.  
3. Constructs a command string to send the Robotino to the specified position.  

### **Returns**  
- **Command string**: The formatted instruction to move the Robotino.  
- **Empty string**: If the Robotino is not found.  

### **Output**  
Prints the movement command to the console.  


In [ ]:
def send_robot_to_position(robot_id, target_x, target_y):
    """
    Sends the robot to a specific position.
    """
    global JobId
    if robot_id not in fleet_state:
        print(f"Robot ID {robot_id} not found in fleet state.")
        return ""

    JobId += 1
    command = f"PushJob GotoPosition {JobId} 1 {robot_id} {target_x} {target_y}\n"
    print(f"Sending robot {robot_id} to position: {command}")
    return command


## **Function: process_fleet_state_response**  
### **Purpose**  
Parses and updates the **fleet_state** based on incoming fleet state data.  

### **Parameters**  
- **data**: A string containing the fleet state information.  

### **Logic**  
1. Checks if the **FleetState** keyword is in the data.  
2. Splits the data into individual Robotino details.  
3. Processes each Robotino's attributes, updating their:  
   - Position (`x`, `y`, `phi`)  
   - Battery state  
   - Status flags (e.g., charging, emergency)  
4. Updates the global **fleet_state** dictionary with parsed details.  

### **Returns**  
- None. The function directly updates the global state.  

### **Output/Logging**  
Prints the updated fleet state or logs errors during processing.  


In [ ]:
def process_fleet_state_response(data):
    """
    Processes the fleet state response to extract detailed robot information.
    """
    global fleet_state

    if "FleetState" not in data:
        return

    try:
        robots_data = data.replace("FleetState ", "").split(" , ")

        for robotino in robots_data:
            attributes = robotino.split()
            robot_info = {}
            robot_id = None

            for attr in attributes:
                key, value = attr.split(":")
                if key == "robotinoid":
                    robot_id = int(value)
                elif key in {"x", "y", "phi", "current"}:
                    robot_info[key] = float(value)
                elif key in {"charging", "laserwarning", "lasersafety", "emergency", "boxpresent"}:
                    robot_info[key] = bool(int(value))
                elif key in {"state", "ipaddress"}:
                    robot_info[key] = value
                elif key == "batteryvoltage":
                    try:
                        robot_info["battery_state"] = convert_voltage_to_percentage(voltage=float(value), 
                                                                                    robotino_id=robot_id)
                    except ValueError as e:
                        logging.error("Robotino ID was unexpectedly not set in process_fleet_state: " + str(e))

            if robot_id is not None:
                fleet_state[robot_id] = robot_info

        print("Updated fleet state:", fleet_state)
    except Exception as e:
        print(f"Error processing fleet state data: {e}")


## **Function: convert_voltage_to_percentage**  
### **Purpose**  
Converts the battery voltage of a Robotino into a percentage based on its model type.  

### **Parameters**  
- **voltage**: The measured voltage of the Robotino's battery.  
- **robotino_id**: The unique ID of the Robotino whose battery is being processed.  

### **Logic**  
1. Determines the Robotino model type from **robotino_configurations**.  
2. Sets model-specific voltage thresholds:  
   - **Model 3**: Minimum = 18.5V, Maximum = 24.5V.  
   - **Model 4**: Minimum = 15.8V, Maximum = 20.6V.  
3. Calculates the percentage using the formula:  
   \[
   \text{percentage} = \frac{\text{voltage} - \text{min_voltage}}{\text{max_voltage} - \text{min_voltage}} \times 100
   \]  

### **Returns**  
- **Battery percentage**: An integer value representing the battery level.  

### **Error Handling**  
Raises a **ValueError** if the Robotino ID is missing.  


In [ ]:
def convert_voltage_to_percentage(voltage, robotino_id):
    """
    Converts voltage of a Robotino's batteries to percentage.
    """
    if robotino_id is None:
        raise ValueError("robotino_id is None")

    if robotino_configurations[robotino_id]['type'] == 3:
        # Correct voltage min/max for model 3
        max_voltage = 24.5
        min_voltage = 18.5
    else:
        # Correct voltage min/max for model 4
        max_voltage = 20.6
        min_voltage = 15.8

    # Calculate battery percentage
    voltage_range = max_voltage - min_voltage
    battery_percentage = int(((voltage - min_voltage) / voltage_range) * 100)
    return battery_percentage


## **Function: handle_incoming_messages**  
### **Purpose**  
Processes messages received from the client, including fleet state updates.  

### **Parameters**  
- **conn**: The socket connection object for communication.  
- **addr**: The address of the connected client.  

### **Logic**  
1. Waits for incoming messages through the socket connection.  
2. Decodes and processes each message:  
   - If the message contains **FleetState**, calls **process_fleet_state_response**.  
3. Handles disconnection or errors gracefully.  

### **Returns**  
- None. Operates as a continuous loop for real-time communication.  

### **Output**  
Prints connection status, received messages, and error details to the console.  


In [ ]:
def handle_incoming_messages(conn, addr):
    """
    Handles incoming messages from the client.
    """
    print('Connected to', addr)

    while True:
        try:
            data = conn.recv(MAX_BUFFER_SIZE)
            if not data:  # Connection closed
                print('Connection closed by', addr)
                break
            decoded_data = data.decode('utf-8').strip()
            print('Received message from', addr, ':', decoded_data)
            # Process the fleet state response
            if "FleetState" in decoded_data:
                process_fleet_state_response(decoded_data)
            # TODO: include other feedback messages (e.g., when a Robotino arrives at a location)
        except Exception as e:
            print(f"Error receiving message from {addr}: {e}")
            break


## **Function: send_fleet_state**  
### **Purpose**  
Maintains a connection with the client by periodically sending fleet state requests.  

### **Parameters**  
- **conn**: The socket connection object for communication.  

### **Logic**  
1. Constructs the **GetFleetState** message.  
2. Sends the message every **2 seconds** to the connected client.  
3. Handles communication errors and exits the loop if any occur.  

### **Returns**  
- None. Operates as a continuous loop for periodic requests.  

### **Output**  
Prints a confirmation message each time a request is sent or an error occurs.  


In [ ]:
def send_fleet_state(conn):
    """
    Sends the "GetFleetState" message every 2 seconds to maintain the connection.
    """
    while True:
        try:
            message_to_send = "GetFleetState\n"
            conn.sendall(message_to_send.encode('utf-8'))
            print("Sent: GetFleetState")
            time.sleep(2)
        except Exception as e:
            print(f"Error sending fleet state: {e}")
            break


## **Function: charging_management**  
### **Purpose**  
Manages the Robotino charging process, ensuring an optimal flow between charging and operational states.  

### **Parameters**  
- **conn**: The socket connection object for communication.  

### **Logic**  
1. Logs the current operational queue and fleet state.  
2. Identifies Robotinos with low battery from the **operational_queue**.  
3. Ensures that only a limited number of Robotinos are assigned chargers based on availability.  
4. Sends commands to Robotinos to move to the nearest chargers and dock.  
5. Monitors charged Robotinos, moving them back to the operational queue.  

### **Returns**  
- None. Operates as a continuous loop for real-time charging management.  

### **Logging**  
Logs the states and transitions of Robotinos, including errors during communication.  


In [ ]:
def charging_management(conn):
    """
    Manages the charging process for Robotinos.
    Ensures only a limited number of Robotinos are charging simultaneously
    and puts Robotinos back to work when charged.
    """
    global JobId
    global robots_moving_to_charger

    while True:
        # Log operational queue state for debugging
        logging.info(f"Operational queue: {operational_queue}")
        logging.info(f"Fleet state: {fleet_state}")

        # Filter Robotinos with low battery in the operational queue
        low_battery_robotinos = [
            robot_id for robot_id in operational_queue
            if robot_id in fleet_state and fleet_state[robot_id]['battery_state'] <= BATTERY_MINIMUM_PERCENT
        ]

        # Log any robot IDs missing in the fleet_state
        for robot_id in operational_queue:
            if robot_id not in fleet_state:
                logging.warning(f"Robotino {robot_id} not found in fleet_state.")

        if not low_battery_robotinos:
            logging.info("No Robotino in the operational queue requires charging.")
            time.sleep(5)  # Wait before checking again
            continue

        logging.info(f"Robotinos needing charge: {low_battery_robotinos}")

        # Ensure only a limited number of chargers are in use
        current_charging_count = len(robots_moving_to_charger)
        available_charger_slots = 3 - current_charging_count

        if available_charger_slots <= 0:
            logging.info("All chargers are currently in use.")
            time.sleep(5)  # Wait before checking again
            continue

        # Limit the number of Robotinos sent for charging
        low_battery_robotinos = low_battery_robotinos[:available_charger_slots]

        # Assign chargers and send Robotinos to charge
        for robot_id in low_battery_robotinos:
            if robot_id in robots_moving_to_charger:
                logging.debug(f"Robotino {robot_id} is already being sent to a charger.")
                continue

            message_to_send = send_robot_to_closest_charger(robot_id)
            if not message_to_send:
                logging.info(f"No chargers available for Robotino {robot_id}.")
                continue

            logging.info(f"Assigning Robotino {robot_id} to charger.")
            try:
                # Send Robotino to the charger
                conn.sendall(message_to_send.strip().encode('utf-8'))
                # Send docking command immediately
                message_to_send = send_robot_to_dock(robot_id)
                conn.sendall(message_to_send.strip().encode('utf-8'))

                # Mark Robotino as moving to charger
                robots_moving_to_charger.append(robot_id)

                # Remove from operational queue and add to charging queue
                operational_queue.remove(robot_id)
                charging_queue.append(robot_id)

            except Exception as e:
                logging.error(f"Error sending message for Robotino {robot_id}: {e}")
                break

        # Check if any Robotinos have completed charging
        charged_robotinos = [
            robot_id for robot_id in charging_queue
            if robot_id in fleet_state and fleet_state[robot_id]['battery_state'] > BATTERY_MINIMUM_PERCENT
        ]

        for robot_id in charged_robotinos:
            logging.info(f"Robotino {robot_id} has completed charging.")
            charging_queue.remove(robot_id)
            operational_queue.append(robot_id)

            # Remove from the moving to charger list
            if robot_id in robots_moving_to_charger:
                robots_moving_to_charger.remove(robot_id)

        time.sleep(5)  # Wait before the next iteration


## **Function: monitor_robotino_status**  
### **Purpose**  
Continuously monitors and prints the status of all Robotinos.  

### **Parameters**  
- **active_robotinos**: List of Robotinos currently performing tasks.  
- **inactive_robotinos**: List of Robotinos not assigned tasks.  
- **drained_robotinos**: List of Robotinos with depleted batteries.  

### **Logic**  
1. Prints categorized statuses of Robotinos every **120 seconds**.  
2. Provides real-time visibility into the fleet's state.  

### **Returns**  
- None. Operates as a continuous monitoring loop.  

### **Output**  
Displays the status of Robotinos in the console.  


In [ ]:
def monitor_robotino_status(active_robotinos, inactive_robotinos, drained_robotinos):
    """
    Function to monitor and print Robotino statuses
    :param active_robotinos: list of robotinos that are doing tasks
    :param inactive_robotinos: list of robotinos that are not doing tasks
    :param drained_robotinos: list of robotinos that the battery has been drained
    """
    while True:
        print("\n=== Robotino Status Update ===")
        print(f"Active Robotinos: {active_robotinos}")
        print(f"Inactive Robotinos: {inactive_robotinos}")
        print(f"Drained Robotinos: {drained_robotinos}")
        print("=============================\n")
        time.sleep(120)


## **Function: main**  
### **Purpose**  
The entry point of the script, responsible for initializing the server and managing Robotino interactions.  

### **Logic**  
1. **Initialize State**  
   - Categorizes Robotinos into `active`, `inactive`, and `drained` states.  
   - Sets up **operational_queue** and **charging_queue** for task and charging management.  
2. **Start Server**  
   - Creates a socket server bound to the specified host and port.  
   - Listens for incoming client connections.  
3. **Handle Connections**  
   - For each client, spawns threads to:  
     - Handle incoming messages via **handle_incoming_messages**.  
     - Periodically send fleet state requests via **send_fleet_state**.  
     - Manage the Robotino charging process via **charging_management**.  

### **Error Handling**  
- Catches and logs errors while accepting client connections.  

### **Returns**  
- None. Operates indefinitely as a server process.  

### **Output**  
- Displays initialization details and connection logs.  


In [ ]:
def main():
    """
    Main function to start the server, accept clients, and create threads for handling messages and sending messages.
    """
    # Initialize lists to categorize Robotinos
    active_robotinos = []
    inactive_robotinos = []
    drained_robotinos = []

    # Initialize queues for operational and charging Robotinos
    operational_queue = [20, 21, 24]  # Robotino IDs: 2 version 3 and 1 version 4
    charging_queue = [22, 23, 25]  # Robotino IDs: 2 version 3 and 1 version 4

    print("Initialized Robotino lists and queues:")
    print(f"Active Robotinos: {active_robotinos}")
    print(f"Inactive Robotinos: {inactive_robotinos}")
    print(f"Drained Robotinos: {drained_robotinos}")
    print(f"Operational Queue: {operational_queue}")

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()
        print(f"Server running and listening on {HOST}:{PORT}")

        while True:
            try:
                conn, addr = s.accept()
                print(f"Client connected from {addr}")

                # Start threads for handling client interaction
                threading.Thread(target=handle_incoming_messages, args=(conn, addr)).start()
                threading.Thread(target=send_fleet_state, args=(conn,)).start()
                threading.Thread(target=charging_management, args=(conn,)).start()
            except Exception as e:
                print("Error accepting connection:", e)


## **Script Execution**  
### **Purpose**  
Defines the entry point for the script to initialize the fleet management process and handle Robotino interactions, such as charging and operational task assignments.

### **Logic**  
1. Executes the main function when the script is run directly. 
2. Includes mechanisms for handling Robotino communication, fleet state management, and charger assignments.
3. Updates to charging logic now include:
Sending repeated commands (sendall) for both moving to the charger and docking.
Adding a slight delay (time.sleep(0.2)) between commands to ensure proper processing.

### **Output**  
- Logs detailed messages about Robotino charger assignments and fleet state updates.
Final output includes:
The current state of the charging_queue to indicate which Robotinos are actively charging.
The state of the operational_queue for Robotinos ready to be assigned new tasks.  


In [ ]:
if __name__ == '__main__':
    try:
        main()
    except Exception as e:
        logging.error(f"An error occurred during script execution: {e}")
    finally:
        print("\n--- Final Fleet States ---")
        print(f"Charging Queue: {charging_queue}")
        print(f"Operational Queue: {operational_queue}")



## **Note on Handling Robotino Fleet State**  
1. **Robotino Not Found**:  
   - If the app shows a Robotino as "not found" in **fleet_state**, it might indicate a defect.  
   - In the future, consider adding a condition to automatically move such Robotinos from the **operational_queue** to a **non_operational list** for better management.  

2. **Battery Percent Simplification**:  
   - The logic for battery percentage at **fleet_state line 395** is simplified.  
   - Be aware that it might not work as intended because the battery percentage is immediately set to 100% once a Robotino starts charging.  
   - This could lead to misinterpretation of its actual charging status.
